In [ ]:
import pathlib
import requests
import urllib
import boto3
import botocore

In [ ]:
SCHEME = "https"
NETLOC = "portal.opentopography.org"
PATH = "/API/otCatalog"

In [ ]:
params = {
        "productFormat": "PointCloud",
        "minx": -117.4,
        "miny": 33.4,
        "maxx": -115.5,
        "maxy": 34.9,
        "detail": False,
        "outputFormat": "json",
        "inlcude_federated": True
    }
data_info_stub = 'https://portal.opentopography.org/datasetMetadata?otCollectionID='

In [ ]:
data_url = urllib.parse.urlunparse((SCHEME, NETLOC, PATH, "", "", ""))

response = requests.get(data_url, params=params, stream=True)
response.raise_for_status()

In [ ]:
json_response

In [ ]:
json_response=response.json()
id_name=json_response['Datasets'][0]['Dataset']['identifier']['value']

data_info_stub + id_name.replace('OTLAS', 'OT')

In [ ]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
USER_AGENT = 'Mozilla/5.0 (iPad; U; CPU OS 3_2_1 like Mac OS X; en-us) AppleWebKit/531.21.10 (KHTML, like Gecko) Mobile/7B405'
#opener = urllib.request.URLopener()
#opener.addheader('User-Agent', USER_AGENT)
#filename, headers = opener.retrieve(tile_url, self.cache_path / pathlib.Path(tile_name))
        

# Try download file - get permission denied

In [ ]:
headers = {'User-Agent': USER_AGENT}
tile_url = "https://opentopography.s3.sdsc.edu/minio/pc-bulk/Wellington_2013/Wellington_2013_TileIndex.zip"
dest_path = r"C:\Users\pearsonra\Documents\data\Wakanae\local_cache\test.zip"
r = requests.get(tile_url, headers=headers, stream=True)
with open(dest_path, 'wb') as fd:
    for chunk in r.iter_content(): #chunk_size=chunk_size
        fd.write(chunk)

# AWS CLI that work
` C:\Users\pearsonra>aws s3 ls s3://pc-bulk/Wellington_2013/Wellington_2013_TileIndex.zip --recursive --endpoint-url https://opentopography.s3.sdsc.edu --no-sign-request`

`C:\Users\pearsonra>aws s3 cp s3://pc-bulk/Wellington_2013/Wellington_2013_TileIndex.zip . --recursive --endpoint-url https://opentopography.s3.sdsc.edu --no-sign-request`
# Notes on possible reason
Typically when you see a 403 on HeadObject despite having the s3:GetObject permission, it's because the s3:ListObjects permission wasn't provided for the bucket AND your key doesn't exist. It's a security measure to prevent exposing information about what objects are or aren't in your bucket. When you have both the s3:GetObject permission for the objects in a bucket, and the s3:ListObjects permission for the bucket itself, the response for a non-existent key is a 404 "no such key" response. If you only have s3:GetObject permission and request a non-existent object, the response is a 403 "access denied".
https://stackoverflow.com/questions/59615713/trouble-downloading-s3-bucket-objects-through-boto3-error-403-headobject-forbi

# AWS keys

In [ ]:
aws_access_key_id = 'pc-bulk'
aws_secret_access_key = 'pc-bulk-3LoTa4'
aws_endpoint_url = 'https://opentopography.s3.sdsc.edu'

aws_bucket = 'Wellington_2013'


tile_file_name = 'Wellington_2013_TileIndex.zip'

ex_full_url = 'https://opentopography.s3.sdsc.edu/minio/pc-bulk/Wellington_2013'

# Boto client - sucess
Updated structure to avoid 403

In [ ]:
client = boto3.client('s3',
                      endpoint_url=aws_endpoint_url,
                      config=botocore.config.Config(signature_version=botocore.UNSIGNED))

In [ ]:
paginator = client.get_paginator('list_objects')
result = paginator.paginate(Bucket=aws_access_key_id, Delimiter='/')
for prefix in result.search('CommonPrefixes'):
    print(prefix.get('Prefix'))

In [ ]:
response = client.list_objects_v2(Bucket=aws_access_key_id, Prefix=aws_bucket)
response_meta = response.get('ResponseMetadata')

In [ ]:
keys = []
if response_meta.get('HTTPStatusCode') == 200:
    contents = response.get('Contents')
    if contents == None:
        print("No objects are available for %s" % aws_bucket)
    else:
        for obj in contents:
            keys.append(obj.get('Key'))
        print("There are %s objects available for %s\n--" % (len(keys), aws_bucket))
        for k in keys:
            print(k)
else:
    print("There was an error with your request.")


In [ ]:
metadata_key = aws_bucket + '/' + tile_file_name
client.download_file(aws_access_key_id, metadata_key, tile_file_name)

# Boto code examples - with 403
### session

In [ ]:
session = boto3.session.Session(aws_access_key_id=aws_access_key_id,
                                aws_secret_access_key=aws_secret_access_key)



s3 = session.resource('s3', endpoint_url=aws_endpoint_url, 
                      config=botocore.config.Config(signature_version=botocore.UNSIGNED))
#bucket.download_file(tile_file_name, 'test.zip')

### Client

In [ ]:
client = boto3.client('s3',
                  aws_access_key_id=aws_access_key_id,
                  aws_secret_access_key=aws_secret_access_key,
                  endpoint_url=aws_endpoint_url,
                  config=botocore.config.Config(signature_version=botocore.UNSIGNED))

#client.download_file(aws_bucket, tile_file_name, 'test.zip')

### Resource

In [ ]:
s3 = boto3.resource('s3', 
                    aws_access_key_id=aws_access_key_id, 
                    aws_secret_access_key=aws_secret_access_key,
                    endpoint_url=aws_endpoint_url,
                    config=botocore.config.Config(signature_version=botocore.UNSIGNED))



#s3.Bucket(aws_bucket).download_file(tile_file_name, 'test.zip')